In [19]:
import numpy as np

# structure
# [datawidth] columns of height data -100 for null
# hole present (0=no or 1=yes)
# hole size (0 for not present, 1 for one pixel wide hole)


# build examples
dataWidth = 6
minSlope = -3
maxSlope = 3
nullValue = -100

# build varying slopes
def generateSlopes():
    dataset = None
    i=0
    slope = np.empty(maxSlope-minSlope+1)
    for currentSlope in range (minSlope, maxSlope+1):
        slope[i] = currentSlope
        lastValue = 0
        if (currentSlope == 0):
            newdata = np.zeros(dataWidth)
        else:
            newdata = np.arange(0, dataWidth*currentSlope, currentSlope)
        if (dataset is None):
            dataset = newdata
        else:
            dataset = np.vstack((dataset, newdata))
        i+=1
    return dataset    

def knockOutHoles(dataset):
    ds1 = dataset.copy()
    ds2 = dataset.copy()
    ds3 = dataset.copy()
    for row in ds1:
        row[2] = nullValue
    for row in ds2:
        row[3] = nullValue
    for row in ds3:
        row[2:4] = nullValue
    return np.concatenate((ds1, ds2, ds3))

def trimLeftEdge(dataset):
    for row in dataset:
        row[0] = nullValue
    return dataset

def trimRightEdge(dataset):
    for row in dataset:
        row[-1] = nullValue
    return dataset

def holeData(row):
    lastValue = nullValue
    steppedDown = False
    holeSize = 0
    for value in row:
        # check if we stepped down to null
        if (steppedDown):
            holeSize = holeSize+1

            # check for step up after having stepped down
            if (lastValue == nullValue and value != nullValue):
                return True, holeSize
                steppedDown = False
        else:
            # check for step down to null
            if (lastValue != nullValue and value == nullValue):
                steppedDown = True

        lastValue = value

    # no hole found
    return False, 0

def buildLabels(unlabled):
    nRows = np.shape(unlabled)[0]
    holePresent = np.zeros((nRows,1))
    holeSize = np.zeros((nRows,1))
    i = 0
    for row in unlabled:
        h = holeData(row)
        holePresent[i] = h[0]
        holeSize[i] = h[1]           
        i = i+1
    lables = np.hstack((holePresent,holeSize))
    return lables


ds = generateSlopes()
ds = np.concatenate((ds, knockOutHoles(ds.copy())))
ds = np.concatenate((ds, trimLeftEdge(ds.copy())))
ds = np.concatenate((ds, trimRightEdge(ds.copy())))

np.shape(ds)

lables = buildLabels(ds)
lds = np.hstack((ds, lables))
# format output
headerStr = ""
for i in range(dataWidth):
    headerStr = headerStr + "v" + i + ", "
headerStr = headerStr + " holePresent, holeWidth"
print("HeaderStr: "+headerStr)
np.savetxt("hole-dataset.csv", lds, delimiter=",", header=headerStr)
print(lds)

SyntaxError: invalid syntax (<ipython-input-19-9aff74c552df>, line 105)